# ImgProcess

Get the GPS coordinates and sync the images

In [1]:
from exif import Image
from datetime import datetime
import re
import pickle

In [2]:
files = !ls img_walnutst/
files

['IMG_20230914_115708259_HDR.jpg',
 'IMG_20230914_115813496_HDR.jpg',
 'IMG_20230914_115829542_HDR.jpg']

In [9]:
s3Loc = "https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/"

In [3]:
def convertGPS(cord):
    outCord = cord[0]
    
    outCord += cord[1] / 60.0
    
    outCord += cord[2] / 3600.0
    
    return outCord

In [13]:
def getMetaData(files, reg = re.compile('^IMG_([^_]+)'), s3Loc = s3Loc):

    outDict = {}
    for file in files:
        with open(f"./img_walnutst/{file}", 'rb') as fileObj:
            img = Image(fileObj)

            lat = convertGPS( img.gps_latitude )
            lon = convertGPS( img.gps_longitude ) * -1.0


            timeStr = reg.match(files[1]).groups()[0]

            date = datetime.strptime(timeStr, '%Y%m%d')

            outDict[s3Loc + file] = (date, (lat,lon))

    return outDict
              
        
        


In [14]:
!aws s3 sync --delete ./img_walnutst s3://filestogeaux.garyscorner.net/pub/img_walnutst/ --acl public-read

In [15]:
metaData = getMetaData(files) 
metaData

{'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_115708259_HDR.jpg': (datetime.datetime(2023, 9, 14, 0, 0),
  (29.931499472222225, -90.12840269444443)),
 'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_115813496_HDR.jpg': (datetime.datetime(2023, 9, 14, 0, 0),
  (29.93270491666667, -90.12771605555555)),
 'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_115829542_HDR.jpg': (datetime.datetime(2023, 9, 14, 0, 0),
  (29.93270491666667, -90.12771605555555))}

In [16]:
with open('img_walnutstMetaData.pickle', 'wb') as f:
    pickle.dump(metaData,f)

In [17]:
str(datetime.now())

'2023-09-14 22:23:55.411819'